# Drugi projekt. 

## Gry Stackelberga

W gry Stackelberga gra lider (obrońca), który wybiera swoją strategię (mieszaną) jako pierwszy i naśladowcy (atakujący), którzy wybierają swoje strategie po poznaniu strategii lidera. Nie są to gry o sumie zerowej: lider i naśladowcy mają swoje własne macierze wypłat.
    
Rozważmy przykład - grę z jednym naśladowcą. Macierz wypłat to

$$
\begin{array}{|c|c|c|}
\hline & c & d \\
\hline a & 2,1 & 4,0\\
\hline b & 1,0 & 3,2\\
\hline
\end{array}
$$

Lider gra wierszami. Jego ruchy to `a` i `b`. 
Naśladowca gra kolumnami. 
Jego ruchy to `c` i `d`.
Pierwsza z liczb w macierzy wypłat to wypłata lidera. Druga to wypłata naśladowcy.

Jedyną równowagą Nasha (przy jednoczesnym wyborze strategii) w tej grze jest zagranie `a` przez lidera i `c` przed naśladowcę, co daje liderowi wypłatę równą $2$.

Jeżeli jednak lider wybierze swoją strategię jako pierwszy i będzie grał `a` i `b` z prawdopodobieństwem $\frac 12$, to naśladowca zmieni swoją strategię na `d`, powiększając (oczekiwaną) wypłatę lidera do $3\frac 12$.

W grze Stackelberga naśladowca zna mieszaną strategię lidera, ale nie zna konkretnego ruchu przez niego wybranego.

## Bayesowskie gry Stackelberga

W bayesowskiej grze Stackelberga dopuszczamy $N$ różnych naśladowców i jednego lidera.
Podczas rozgrywki jako pierwszy strategię wybiera lider, następnie swoje strategie wybierają naśladowcy.
Rozgrywka jest pomiędzy liderem a jednym z naśladowców (nazwijmy go $n$), wybieranym z puli z prawdopodobieństwem $p_n$.

Rozszerzając poprzedni przykład
$$
\begin{array}{|c|c|c|}
\hline & c & d \\
\hline a & 2,1 & 4,0\\
\hline b & 1,0 & 3,2\\
\hline
\end{array}
$$
możemy dodać drugiego naśladowcę
$$
\begin{array}{|c|c|c|}
\hline & c' & d' \\
\hline a & 1,1 & 2,0\\
\hline b & 0,1 & 3,2\\
\hline
\end{array}
$$
Każdy z naśladowców pojawia się z prawdopodobieństwem $\frac 12$.

## Wyznaczanie optymalnej strategii za pomocą MILP

Zapiszemy problem wyznaczania optymalnej strategii lidera jako problem całkowitoliczbowy.
Zaczniemy od naturalnego sformułowania problemu z nieliniowymi (kwadratowymi) warunkami.
Optymalna strategia lidera zakłada, że naśladowcy dobierają optymalne strategie do jego strategii.
W takim przypadku naśladowcy zawsze mogą wybrać strategie czyste - zawsze jeden z ruchów będzie nie gorszy od pozostałych.

Zbiór typów naśladowców oznaczamy przez $L$.
Prawdopodobieństwo gry przeciwko naśladowcy typu $l \in L$ oznaczamy przez $p^l$.
Jest ono z góry dane.

Przez $x$ oznaczamy strategię lidera. Liczba  $x_i$ oznacza prawdopodobieństwo zagrania ruchu $i$ przez lidera.
Strategię naśladowcy typu $l \in L$ oznaczamy przez $q^l$.
Przez $R^l$ i $C^l$ oznaczamy wypłaty dla lidera i naśladowcy w grze pomiędzy liderem a naśladowcą typu $l$.
Przez $X$ i $Q$ oznaczamy możliwe ruchy (strategie czyste) lidera i naśladowców.

Niech $M$ oznacza (dostatecznie) dużą liczbę.

Strategię lidera możemy wyznaczyć rozwiązując następujący problem.

$$
\max_{x,q,a} \sum_{i \in X} \sum_{l \in L} \sum_{j \in Q} p^l R^l_{ij} x_i q^l_j
$$
pod warunkami
$$
\sum_{i\in X} x_i = 1
$$
$$
\sum_{j \in Q} q^l_j = 1
$$
$$
0 \leq (a^l - \sum_{i \in X} c^l_{ij} x_i) \leq (1 - q^l_j) M
$$
$$
x_i \in [0, 1]
$$
$$
q^l_j \in \{0,1\}
$$
$$
a^l \in \mathbb{R}
$$

## Zadanie - część teoretyczna

Wyjaśnij dlaczego rozwiązanie optymalne powyższego problemu wyznacza strategię optymalną lidera. Co to jest $a^l$ w problemie?

Zwróć uwagę, że problem nie jest liniowy: w funkcji celu mamy iloczyny $x_i q^l_j$ a zmienne $q^l_j$ są binarne.

Następnie udowodnij, że następująca *linearyzacja* (zmienne binarne pozostają) problemu jest poprawna:
$$
\max_{q,z,a} \sum_{i\in X}\sum_{l \in L}\sum_{j \in Q} p^l R^l_{ij} z^l_{ij}
$$
pod warunkami
$$
\sum_{i \in X} \sum_{j \in Q} z^l_{ij} = 1
$$
$$
\sum_{j \in Q} z^l_{ij} \leq 1
$$
$$
q^l_j \leq \sum_{i\in X} z^l_{ij} \leq 1
$$
$$
\sum_{j \in Q}q^l_j = 1
$$
$$
0 \leq (a^l - \sum_{i \in X} C^l_{ij} (\sum_{h \in Q} z^l_{ih})) \leq (1 - q^l_j) M
$$
$$
\sum_{j \in Q} z^l_{ij} = \sum_{j \in Q} z^1_{ij}
$$
$$
z^l_{ij} \in [0, 1]
$$
$$
q^l_j \in \{0,1\}
$$
$$
a^l \in \mathbb{R}
$$

Jest to tak zwany algorytm DOBSS, pochodzący z pracy 

`Paruchuri et al., Efficient Algorithms to Solve Bayesian Stackelberg Games for Security
Applications, AAAI 2008.`

## Zadanie - część praktyczna

Gry Stackelberga w ostatnich latach były stosowane w wielu kontekstach praktycznych. Poczynając od ochrony lotniska LAX w Los Angeles, poprzez optymalizację działań patroli USCG (U.S. Coastal Guard) po ochronę dzikich zwierząt przed kłusownikami w rezerwatach i wyłapywanie w metrze pasażerów na gapę.

Przykład z pracy (cytuję sekcję 4.3)

`Pita et al., Deployed ARMOR Protection: The Application of a Game
Theoretic Model for Security at the Los Angeles
International Airport, AAMAS 2008.`

---

> We now illustrate how the security problems set forth by LAWA police, i.e. where and when to deploy checkpoints and canines, can be cast in terms of a Bayesian Stackelberg game. We focus on the checkpoint problem for illustration, but the case of the canine problem is similar. At LAX, there are a specific number of inbound roads on which to set up checkpoints, say roads 1 through n, and LAWA police have to pick a subset of those roads to place checkpoints on prior to adversaries selecting which roads to attack. We assume that there are m different types of adversaries, each with different attack capabilities, planning constraints, and financial ability. Each adversary type observes the LAWA-police check-point policy and then decides where to attack. Since adversaries can observe the LAWA police policy before deciding their actions, this situation can be modeled via a Stackelberg game with the police as the leader.
>
> In this setting the set X of possible actions for LAWA police is the set of possible checkpoint combinations. If, for instance, LAWA police were setting up one checkpoint then $X = \{ 1, . . . , n \}$. If LAWA police were setting up a combination of two checkpoints, then $X = \{(1, 2), (1, 3)...(n − 1, n)\}$, i.e. all combinations of two checkpoints. Each adversary type $l \in L = \{1, . . . , m\}$ can decide to attack one of the n roads or maybe not attack at all (none), so its set of actions is $Q = \{1, . . . , n, none\}$. If LAWA police select
road i to place a checkpoint on and adversary type $l \in L$ selects road j to attack then the agent receives a reward $R^l_{ij}$ and the adversary receives a reward $C^l_{ij}$. These reward values vary based on three considerations: 
>
>(i) the chance that the LAWA police check-point will catch the adversary on a particular inbound road; 
>
>(ii) the damage the adversary will cause if it attacks via a particular in-
bound road; 
>
>(iii) type of adversary, i.e. adversary capability. 
>
>If LAWA police catch the adversary when i = j we make $R^l_{ij}$ a large positive value and $C^l_{ij}$ a large negative value. However, the probability of catching the adversary at a checkpoint is based on the volume of traffic through the checkpoint (significant traffic will increase the difficulty of catching the adversary), which is an input to the system. If the LAWA police are unable to catch the adversary, then the adversary may succeed, i.e. we make $R^l_{ij}$ a large negative value and $C^l_{ij}$ a large positive value. Certainly, if the adversary attacks via an inbound road where no checkpoint was set up, there is no chance that the police will catch the adversary. The magnitude of $R^l_{ij}$ and $C^l_{ij}$ vary based on the adversary’s potential target, given the road from which the adversary attacks. Some roads lead to higher valued targets for the adversary than others. The game is not a zero sum game however, as even if the adversary is caught, the adversary may benefit due to publicity.
>
>The reason we consider a Bayesian Stackelberg game is because LAWA police face multiple adversary types. Thus, differing values of the reward matrices across the different adversary types $l \in L$ represent the different objectives and valuations of the different attackers (e.g. smugglers, criminals, terrorists). For example, a hard-
core, well-financed adversary could inflict significant damage on LAX; thus, the negative rewards to the LAWA police are much higher in magnitude than an amatuer attacker who may not have sufficient resources to carry out a large-scale attack. Currently we model two types of adversary LAWA may face. A 20-80 split of probability implies that while there is a 20% chance that the LAWA police face the former type of adversary, there is an 80% chance
that they face an amatuer attacker. Our experimental data provides detailed results about the sensitivity of our algorithms to the probability distributions over different adversary types. Given these two adversary types the largest game we have constructed, which was done for canine deployment, consisted of 784 actions for the LAWA
police (when multiple canine units were active) for the eight possible terminals within the airport and 9 actions per adversary type (one for a possible attack on each terminal, and one for none).

---

W części praktycznej Twoim zadaniem jest zaproponowanie scenariusza zastosowania gry Stackelberga: określenie lidera i naśladowców oraz ich macierzy wypłat, a następnie policzynie optymalnej strategii dla lidera.
Na wyższe oceny mile widziana jest kreatywność w sformułowaniu problemu. Na zaliczenie wystarczy odwtorzyć powyższy scenariusz z lotniska LAX.

## Forma rozwiązania i termin

Rozwiązanie powinno zawierać raport zawierający opis problemu, szczegółowy opis przyjętego sposobu rozwiązania, arkusz zawierający wykonane obliczenia oraz uzyskane wyniki (część praktyczna) oraz dowód poprawności (część teoretyczna).

Rozwiązania należy wysyłać e-mailem do prowadzącego ćwiczenia (w grupie piątkowej z kopią do wykładowcy) do 19 czerwca do godz. 23:59. W przypadku problemu z dotrzymaniem terminu proszę o wcześniejszy kontakt z wykładowcą (zanim termin minie!).

Zaliczenie podczas ustnego egzaminu w trakcie sesji. Na egzamin trzeba przygotować prezentację (maksymalnie 10 minut, należy przygotować slajdy) na temat uzyskanych wyników i wykorzystanych metod.